In [ ]:
import pandas as pd

In [ ]:
import sqlite3

In [ ]:
emp = pd.read_csv('/content/employees.csv')

In [ ]:
emp[emp['emp_no']==10010]

In [ ]:
emp_no_list = [10010,10011]
emp[emp['emp_no'].isin(emp_no_list)]

In [ ]:

conn = sqlite3.connect(':memory:')
emp.to_sql('employees', conn, index=False, if_exists='replace') 


In [ ]:
emp.shape

In [ ]:
emp[emp['emp_no'].isin(emp_no_list)]

In [ ]:
salaries = pd.read_csv('/content/Salaries.csv')

In [ ]:
salaries.shape

In [ ]:

salaries[salaries['emp_no'].isin(emp_no_list)]

In [ ]:

salaries.merge(emp,on='emp_no')



In [ ]:
titles = pd.read_csv('/content/titles.csv')

In [ ]:
titles

In [ ]:
titles[titles['emp_no'].isin(emp_no_list)]

In [ ]:
# first name , salary (max ) latest salary , hire date , last working if employees present 9999 current emploe , year ex , titels
#left join it better split query


In [ ]:
query = """
SELECT
    e.first_name,
    e.last_name,
    MAX(s.salary) AS max_salary,
    s.salary AS latest_salary,
    e.hire_date,
    CASE
        WHEN s.to_date = '9999-01-01' THEN 'Current Employee'
        ELSE s.to_date
    END AS last_working_date,
    CASE
        WHEN s.to_date = '9999-01-01' THEN strftime('%Y', 'now') - strftime('%Y', e.hire_date)
        ELSE strftime('%Y', s.to_date) - strftime('%Y', e.hire_date)
    END AS years_experience,
    t.title
FROM employees e
LEFT JOIN (
    SELECT emp_no, salary, from_date, to_date
    FROM salaries
    WHERE to_date = '9999-01-01' OR to_date = (
        SELECT MAX(to_date)
        FROM salaries AS s2
        WHERE s2.emp_no = salaries.emp_no
    )
) s ON e.emp_no = s.emp_no
LEFT JOIN (
    SELECT emp_no, title, from_date, to_date
    FROM titles
    WHERE to_date = '9999-01-01' OR to_date = (
        SELECT MAX(to_date)
        FROM titles AS t2
        WHERE t2.emp_no = titles.emp_no
    )
) t ON e.emp_no = t.emp_no
GROUP BY e.emp_no
ORDER BY e.first_name;
"""

# Execute the query
df = pd.read_sql_query(query, conn)
print(df)


In [ ]:
# List all tables in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)
